In [1]:
#Sytem
import sys,os
import datetime
import numpy as np
import pandas as pd
import warnings
import time
import pprint
import random
import IPython
warnings.filterwarnings('ignore')

#Visualidation
import sweetviz as sv
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline

#EDA
import sweetviz as sv
import sklearn
from sklearn.preprocessing import OneHotEncoder, LabelEncoder



#model
from sklearn.model_selection import train_test_split,RandomizedSearchCV,GridSearchCV
from skopt import BayesSearchCV
import xgboost as xgb
from xgboost import XGBClassifier




print('-'*40)

print("Python version: {}". format(sys.version))

print("pandas version: {}". format(pd.__version__))

print("matplotlib version: {}". format(matplotlib.__version__))

print("NumPy version: {}". format(np.__version__))


print("scikit-learn version: {}". format(sklearn.__version__))
print('-'*40)

#Info
Target = "Perished"
PATH_Train = '../data/input/train_beta.csv'
PATH_Test = '../data/input/test_beta.csv'
PATH_Sample = '../data/input/gender_submission.csv'
PATH_LOG = "../logs/log.csv"
ID = 12
SEED = 42

----------------------------------------
Python version: 3.9.2 (tags/v3.9.2:1a79785, Feb 19 2021, 13:44:55) [MSC v.1928 64 bit (AMD64)]
pandas version: 1.3.2
matplotlib version: 3.4.3
NumPy version: 1.22.4
scikit-learn version: 0.24.2
----------------------------------------


In [2]:
df_train = pd.read_csv(PATH_Train)
df_test = pd.read_csv(PATH_Test)
sample = pd.read_csv(PATH_Sample)
data = pd.concat([df_train,df_test],sort=False)
print('-'*25)
print("Shape of Train Data\n[row :{},column :{}]".format(
    df_train.shape[0], df_train.shape[1]))
print('-'*25)
print("Shape of Test Data\n[row :{},column :{}]".format(
    df_test.shape[0], df_test.shape[1]))
print('-'*25)


-------------------------
Shape of Train Data
[row :891,column :22]
-------------------------
Shape of Test Data
[row :418,column :22]
-------------------------


In [7]:
data.columns

Index(['PassengerId', 'Perished', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked', 'Title', 'FareBin',
       'Fare_Code', 'Family', 'Age_Code', 'Embarked_Code', 'Title_Code', 'C',
       'Q', 'S'],
      dtype='object')

In [4]:
drop_columns = ["Name","Ticket","Cabin","Embarked","Title",'FareBin',"Fare_Code","Title_Code","Family","Embarked_Code"]
dataset = data.drop(columns=drop_columns)
train = dataset[:len(df_train)]
test = dataset[len(df_train):]
pprint.pprint(dataset.columns.tolist())

['PassengerId',
 'Perished',
 'Pclass',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Fare',
 'Age_Code',
 'C',
 'Q',
 'S']


In [5]:
X = train.iloc[:, 2:].values
y = train.iloc[:, 1].values
X_test = test.iloc[:, 2:].values
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=SEED)

In [6]:
Search = "x"
bestparam = "x"
CV = "x"
Modelname = "XGB"
base_model = XGBClassifier(n_estimators = 10, random_state = SEED)
base_model.fit(X_train, y_train)
print('Train Score: {}'.format(round(base_model.score(X_train, y_train), 4)))
print(' Valid Score: {}'.format(round(base_model.score(X_valid, y_valid), 4)))
STrain = base_model.score(X_train,y_train)
SValid = base_model.score(X_valid,y_valid)

Train Score: 0.8919
 Valid Score: 0.8156


In [7]:
Modelname = "XGB"
model = XGBClassifier(random_state=SEED)
CV = 5
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
max_features = ['auto', 'sqrt']
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]
param_grid = {
    'n_estimators':n_estimators,
    'max_depth':[10],
    'min_samples_split':min_samples_split,
    'min_samples_leaf':min_samples_leaf,
    'max_features':max_features,
}
print(param_grid)

{'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'max_depth': [10], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'max_features': ['auto', 'sqrt']}


In [8]:
Search = "GS"
model_cv = GridSearchCV(estimator=model,param_grid=param_grid,scoring='accuracy',cv=CV,verbose=2,n_jobs=-1)
model_cv.fit(X_train,y_train)
bestparam = str(model_cv.best_params_)

Fitting 5 folds for each of 180 candidates, totalling 900 fits


In [ ]:
Search = "RS"
model_cv = RandomizedSearchCV(estimator=model,param_distributions=param_grid,n_iter=100,cv=CV,verbose=1,random_state=SEED,n_jobs=-1)
model_cv.fit(X_train,y_train)
bestparam = str(model_cv.best_params_)

In [ ]:
Search = "BS"
model_cv= BayesSearchCV(estimator=model,search_spaces=param_grid,n_iter=100,cv=CV,verbose =2)
model_cv.fit(X_train,y_train)
bestparam = str(model_cv.best_params_)

In [ ]:
best_model = model_cv.best_estimator_
print(best_model)
bestparam = str(model_cv.best_params_)
STrain = best_model.score(X_train, y_train)
SValid  = best_model.score(X_valid, y_valid)
print('Train Score: {}'.format(STrain))
print(' Valid Score: {}'.format(SValid))

In [ ]:
log = pd.read_csv(PATH_LOG)
Row  =  len(log)
log.loc[Row] = "x"
log.loc[Row,"Modelname"] = Modelname
log.loc[Row,"Search"] = Search
log.loc[Row,"Vestparams"] = bestparam
log.loc[Row,"CV"] = CV
log.loc[Row,"STrain"] = STrain
log.loc[Row,"SValid"] = SValid
log.loc[Row,"Age_Code_N"] = 3
use_columns = test.columns.drop(["PassengerId","Perished"])
for col in use_columns:
    log.loc[Row,col] = "o"
log.fillna("x",inplace=True)
# if PublicScore is knowed
#log.loc[Row,"SPublic"] = 0.791

In [ ]:
log.to_csv(PATH_LOG,index_label=False)
log.tail()

,Modelname,STrain,SValid,Pclass,Sex,Age,SibSp,Parch,Fare,C,...,Title_Code,Embarked_Code,Search,CV,Bestparams,Title,FareBin,Fare_Code,Vestparams,Family
56,ETC,0.8567415730337079,0.8212290502793296,o,o,o,o,o,o,x,...,o,o,BS,5,x,x,x,o,"OrderedDict([('max_depth', 40), ('max_features...",o
57,RFC,0.9691011235955056,0.7877094972067039,o,o,o,o,o,o,o,...,x,x,x,5,x,x,x,x,x,x
58,RFC,0.9171348314606742,0.8379888268156425,o,o,o,o,o,o,o,...,x,x,BS,5,x,x,x,x,"OrderedDict([('max_depth', 10), ('max_features...",x
59,Ensemble,0.9932659932659933,0.9932659932659933,o,o,o,o,o,o,o,...,x,x,x,x,x,x,x,x,x,x
60,XGB,0.891854,0.815642,o,o,o,o,o,o,o,...,x,x,x,x,x,x,x,x,x,x
